# Import packages and set parameters

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Define methods

## Gap detection in DOS

In [ ]:
@jit(nopython=True)
def gap_detector(fermis, IDS, min_IDS_width = 0.001, min_energy_width = 0.1):
    gaps = np.zeros((0,2),dtype=np.float64)
    
    i = 0
    while i <= IDS.size:

        j = 1
        while abs(IDS[i+j]-IDS[i]) < min_IDS_width:
            if j+i == IDS.size-1:
                break
            else:
                j +=1
        else:
            if abs(fermis[i+j]-fermis[i]) > min_energy_width:
                gaps = np.append(gaps,np.array([[fermis[i],fermis[i+j]]]),axis=0)
                i += j
                continue
            else:
                i +=1
                continue
        
        return gaps

## Chern coefficients

In [ ]:
def n_u1t1(theta,Ch_u1t1,Ch_u1u2t1t2):
    return Ch_u1t1 + theta * Ch_u1u2t1t2

def n_u2t2(theta,Ch_u2t2,Ch_u1u2t1t2):
    return Ch_u2t2 + theta * Ch_u1u2t1t2

def n_u1u2(flux,Ch_u1u2,Ch_u1u2t1t2):
    return Ch_u1u2 - flux * Ch_u1u2t1t2

def n_(theta,flux,Ch_,Ch_u1t1,Ch_u2t2,Ch_t1t2,Ch_u1u2t1t2):
    return Ch_ - flux * Ch_t1t2 - theta * Ch_u1t1 - theta * Ch_u2t2 - theta**2 * Ch_u1u2t1t2

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]

    if "n_fermi" in data:
        df = pd.concat([df,data])
    if "n_moments" in data and not "flux" in data:
        df = pd.concat([df,data])

display(df.set_index('key').sort_index(ascending=False))

## Select data

### skx

In [ ]:
key_flux, key_fermi = '1710151112', '1747815794'

## Import file and extract data

In [ ]:
# parameter
tex, size        = df.set_index('key').at[key_fermi,'texture'],     df.set_index('key').at[key_fermi,'system_sizes']
theta, flux      = df.set_index('key').at[key_fermi,'q'],           df.set_index('key').at[key_fermi,'flux']
sys_size         = df.set_index('key').at[key_flux,'system_sizes']
t,       m       = df.set_index('key').at[key_flux,'t'],           df.set_index('key').at[key_flux,'m']
shift,   mag     = df.set_index('key').at[key_flux,'shift'],       df.set_index('key').at[key_flux,'mag']

n_energies, n_moments, n_random_states = df.set_index('key').at[key_flux,'n_energies'], df.set_index('key').at[key_flux,'n_moments'], df.set_index('key').at[key_flux,'n_random_states'],
scale, epsilon = 12, 0.01,

n_fermi, min_fermi, max_fermi = df.set_index('key').at[key_fermi,'n_fermi'],  df.set_index('key').at[key_fermi,'min_fermi'], df.set_index('key').at[key_fermi,'max_fermi']

# Data
outdir_fermi = './data/' + key_fermi
outdir_flux = './data/' + key_flux

# DOS
fermis = np.linspace(min_fermi,max_fermi,n_fermi)

spectrum = np.load(outdir_fermi+"/spectrum.npy")

# Chern numbers
# New Convention: tau1 = -t1 and order u1u2t1t2
# u1u2         -> u1u2
# tau1u1       -> (-1)^2 u1t1
# tau2u1       -> (-1)^2 u1t2
# tau1u2       -> (-1)^2 u2t1
# tau2u2       -> (-1)^2 u2t2
# tau1tau2     -> (-1)^2 t1t2
# tau1tau2u1u2 -> (-1)^2 u1u2t1t2

ch_0 = np.array([])
for f in fermis:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum])
    ch_0 = np.append(ch_0,[sum(occ)/size**2])

ch_u1u2     = np.load(outdir_fermi+"/ch_u1u2.npy")
ch_u1t1     = np.load(outdir_fermi+"/ch_tau1u1.npy")
ch_u1t2     = np.load(outdir_fermi+"/ch_tau2u1.npy")
ch_u2t1     = np.load(outdir_fermi+"/ch_tau1u2.npy")
ch_u2t2     = np.load(outdir_fermi+"/ch_tau2u2.npy")
ch_t1t2     = np.load(outdir_fermi+"/ch_tau1tau2.npy")
ch_u1u2t1t2 = np.load(outdir_fermi+"/ch_tau1tau2u1u2.npy")

n_0        = n_(theta,flux,ch_0,ch_u1t1,ch_u2t2,ch_t1t2,ch_u1u2t1t2)     # _
n_u1u2     = n_u1u2(flux,ch_u1u2,ch_u1u2t1t2)                            # u1u2
n_u1t1     = n_u1t1(theta,ch_u1t1,ch_u1u2t1t2)                           # u1t1
n_u1t2     = ch_u1t2                                                     # u1t2
n_u2t1     = ch_u2t1                                                     # u2t1
n_u2t2     = n_u2t2(theta,ch_u2t2,ch_u1u2t1t2)                           # u2t2
n_t1t2     = ch_t1t2                                                     # t1t2
n_u1u2t1t2 = ch_u1u2t1t2                                                 # u1u2t1t2 

# orbital magnetisation
morb   = np.load(outdir_fermi+"/morb.npy")
morbLC = np.load(outdir_fermi+"/morbLC.npy")
morbIC = np.load(outdir_fermi+"/morbIC.npy")
    
# data
qs_flux = np.array([i/sys_size for i in range(1,math.floor(sys_size/2))])

emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

dos_flux = []
for i in range(len(qs_flux)):
    dos_flux.append( np.load(outdir_flux+'/dos_'+str(i).zfill(4)+'.npy') )
dos_flux = np.array(dos_flux)

# double range
qs_flux = np.concatenate((qs_flux,np.array([1-q for q in qs_flux[::-1]])))
dos_flux = np.concatenate((dos_flux,dos_flux[::-1]))

Emin = np.amin(emesh)
Emax = np.amax(emesh)

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'CMU serif'

tfs    = 24 #title font size
lfs    = 24 #label font size
fs     = 22 #font size
cbarfs = 18 #colorbar font size
afs    = 24 #annotation font size
gfs    = 18 #gap font size

# scatter size
s = 4

linthresh = 0.07 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale)

# DoS + Chern

In [ ]:
gaps = gap_detector(fermis,ch_0,0.001,0.1)
print(gaps.shape)

In [ ]:
x_ticks1 = np.arange(0.1, 0.5, 0.1)
x_ticks2 = np.arange(0, 3, 1)
x_ticks3 = np.arange(0, 2, 1)
x_ticks4 = np.arange(-4, 6, 2)
x_ticks5 = np.arange(-4, 6, 2)
x_ticks6 = np.arange(-0.3, 0.4, 0.1)
y_ticks = np.arange(-10, 12, 2.5)
    
fig = plt.figure()
gs = fig.add_gridspec(1,32, hspace=0,wspace=0)
ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=12))
ax2 = plt.subplot(gs.new_subplotspec((0, 12), colspan=4),sharey=ax1)
# ax3 = plt.subplot(gs.new_subplotspec((0, 12), colspan=4),sharey=ax1)
ax4 = plt.subplot(gs.new_subplotspec((0, 16), colspan=6),sharey=ax1)
# ax5 = plt.subplot(gs.new_subplotspec((0, 22), colspan=8),sharey=ax1)
ax6 = plt.subplot(gs.new_subplotspec((0, 22), colspan=10),sharey=ax1)

ax1.imshow(dos_flux.T, aspect='auto',norm=norm, extent=(0,1, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax1.set_xticks(x_ticks1)
ax1.set_xlim([0,0.5])
ax1.xaxis.set_minor_locator(MultipleLocator(0.05))
ax1.set_title("DOS",fontsize=tfs)
ax1.set_xlabel(r"$\phi$",fontsize=lfs)
ax1.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)

ax1.axvline(x = flux, color = 'red', label = r'$\flux$')
ax1.annotate(r'$\phi=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(flux*size,size),(flux-0.09,-1.5),color='red',fontsize=afs)

# ax1.annotate(r'$U_{-1}$',(flux+0.005,7.4),color='black',fontsize=gfs)
# ax1.annotate(r'$U_{-2}$',(flux+0.005,6.9),color='black',fontsize=gfs)

# ax1.annotate(r'$U_1$',(flux+0.01,0.06),color='black',fontsize=gfs)
# ax1.annotate(r'$U_2$',(flux+0.05,0.7),color='black',fontsize=gfs)
# ax1.annotate(r'$U_3$',(flux+0.01,1.35),color='black',fontsize=gfs)
# ax1.annotate(r'$U_4$',(flux+0.05,1.8),color='black',fontsize=gfs)
# ax1.annotate(r'$U_5$',(flux+0.01,2.3),color='black',fontsize=gfs)
# ax1.annotate(r'$U_6$',(flux+0.05,2.85),color='black',fontsize=gfs)
# ax1.annotate(r'$U_7$',(flux+0.01,3.2),color='black',fontsize=gfs)
# ax1.annotate(r'$U_8$',(flux+0.05,3.55),color='black',fontsize=gfs)
# ax1.annotate(r'$U_9$',(flux+0.01,3.95),color='black',fontsize=gfs)
# ax1.annotate(r'$U_{10}$',(flux+0.045,4.3),color='black',fontsize=gfs)

# ax1.annotate(r'$C_0$',(flux+0.01,-1.6),color='black',fontsize=gfs)

# ax1.annotate(r'$L_{-1}$',(flux+0.01,-2.6),color='black',fontsize=gfs)

# ax1.annotate(r'$L_1$',(flux+0.01,-10.05),color='black',fontsize=gfs)
# ax1.annotate(r'$L_2$',(flux+0.01,-9.6),color='black',fontsize=gfs)

ax2.scatter(n_0.real,fermis,s)
ax2.set_xticks(x_ticks2)
ax2.xaxis.set_minor_locator(MultipleLocator(0.5))
ax2.grid(True,which='minor',color='lightgray',axis='x')
ax2.set_xlim([-0.5,2.5])
ax2.set_title(r'$n_\emptyset$',fontsize=tfs,y=1.01)

# ax3.scatter(n_u1u2.real,fermis,s)
# ax3.set_xticks(x_ticks3)
# ax3.xaxis.set_minor_locator(MultipleLocator(0.5))
# ax3.grid(True,which='minor',color='lightgray',axis='x')
# ax3.set_xlim([-0.5,1.5])
# ax3.set_title(r'$n_{u_1 u_2}$',fontsize=tfs,y=1.01)

ax4.scatter(n_t1t2.real,fermis,s)
ax4.set_xticks(x_ticks4)
ax4.xaxis.set_minor_locator(MultipleLocator(1))
ax4.grid(True,which='minor',color='lightgray',axis='x')
ax4.set_xlim([-3,6])
ax4.set_title(r'$n_{t_1 t_2}$',fontsize=tfs,y=1.01)

# ax5.scatter(n_u1u2t1t2.real,fermis,s)
# ax5.set_xticks(x_ticks5)
# ax5.xaxis.set_minor_locator(MultipleLocator(1))
# ax5.grid(True,which='minor',color='lightgray',axis='x')
# ax5.set_xlim([-5,5])
# ax5.set_title(r'$n_{u_1 u_2 t_1 t_2}$',fontsize=tfs,y=1.01)

ax6.scatter(morb.real,fermis,s=2,label=r'$M^z_\mathrm{orb}$')
ax6.scatter(morbLC.real-fermis*ch_t1t2.real/(-4*pi),fermis,s=2,c='r',label=r'$M^{LC}_\mathrm{orb}+\frac{E_F}{4\pi} Ch_{t_1 t_2}$')
ax6.scatter(morbIC.real-fermis*ch_t1t2.real/(-4*pi),fermis,s=2,c='g',label=r'$M^{IC}_\mathrm{orb}+\frac{E_F}{4\pi} Ch_{t_1 t_2}$')
ax6.set_xticks(x_ticks6)
ax6.xaxis.set_minor_locator(MultipleLocator(0.02))
ax6.grid(True,which='minor',color='lightgray',axis='x')
ax6.set_xlim([-0.32, 0.32])
ax6.set_title(r'$M^z_\mathrm{orb}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs,y=1.01)
ax6.legend(loc = (0.55,0.455), fontsize= 14, markerscale=4.0)

for ax in fig.get_axes():
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.set_yticks(y_ticks)
    ax.set_ylim([-11,8.5])
    
for ax in fig.get_axes()[1:6]:
    ax.grid(True,color='gray',axis='x')
    #ax.grid(True,which='minor',color='lightgray')
    
    for g in gaps:
        ax.fill_between(np.arange(-15,15,0.01),g[0],g[1], facecolor='gray', alpha=0.5)
        
for g in gaps:
    ax1.fill_between(np.arange(flux-0.001,0.5,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
        
for ax in fig.get_axes()[1:]:
    ax.yaxis.tick_right()

plt.savefig("./plots/flux_Chernhierarchy_kpm_morb.png", dpi=150, bbox_inches = 'tight')